find the dataset [here](https://www.kaggle.com/gpucloud/imdb-wiki)

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input/imdb-db"))

['imdb_db.mat']


In [2]:
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from scipy.io import loadmat
from datetime import datetime

from keras.models import load_model, Model
from keras.utils import np_utils

Using TensorFlow backend.


In [3]:
#Transfer learning
model = load_model("../input/weights/weights.29-3.76_utk.hdf5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [4]:
#last_layer
last_layer = model.get_layer('average_pooling2d_1').output

In [5]:
def load_data(path):
    d = loadmat(path)
    return d["image"], d["gender"][0], d["age"][0], d["db"][0], d["img_size"][0, 0], d["min_score"][0, 0]

In [6]:
image, gender, age, _, image_size, _ = load_data("../input/imdb-db/imdb_db")

In [7]:
X_data = image
y_data_g = np_utils.to_categorical(gender, 2)
y_data_a = np_utils.to_categorical(age, 101)

In [8]:
image_input = Input(shape=(64, 64, 3))
x= Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
#dense layer for gender prediction
predictions_g = Dense(2, activation='softmax', name='gender')(x)

#dense layer for age prediction
predictions_a = Dense(101, activation='softmax', name='age')(x)

In [9]:
#use the output layer for both gender prediction age prediction
model_custom = Model(model.get_layer('input_1').output, [predictions_a, predictions_g])

In [10]:
model_custom.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 16)   432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 64, 64, 16)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [11]:
#train only last layers
for layer in model_custom.layers[:-5]:
    layer.trainable = False

In [12]:
#complie
import keras
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model_custom.compile(loss='categorical_crossentropy',optimizer= 'adam', metrics=['accuracy'])

In [13]:
#train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train_g, y_test_g, y_train_a, y_test_a=train_test_split(X_data, y_data_g,y_data_a, test_size=0.15, random_state=42,shuffle=True)

In [14]:
# train the model
model_custom.fit(X_train, [y_train_a, y_train_g],batch_size=512, epochs=24)

Epoch 1/24
146074/146074 [==============================] - 167s 1ms/step - loss: 3.7524 - age_loss: 3.4267 - gender_loss: 0.2351 - age_acc: 0.0761 - gender_acc: 0.9101
Epoch 2/24
146074/146074 [==============================] - 157s 1ms/step - loss: 3.6173 - age_loss: 3.3006 - gender_loss: 0.2262 - age_acc: 0.0956 - gender_acc: 0.9134
Epoch 3/24
146074/146074 [==============================] - 157s 1ms/step - loss: 3.5804 - age_loss: 3.2668 - gender_loss: 0.2231 - age_acc: 0.1061 - gender_acc: 0.9145
Epoch 4/24
146074/146074 [==============================] - 157s 1ms/step - loss: 3.5531 - age_loss: 3.2419 - gender_loss: 0.2206 - age_acc: 0.1118 - gender_acc: 0.9148
Epoch 5/24
146074/146074 [==============================] - 157s 1ms/step - loss: 3.5342 - age_loss: 3.2250 - gender_loss: 0.2186 - age_acc: 0.1170 - gender_acc: 0.9155
Epoch 6/24
146074/146074 [==============================] - 157s 1ms/step - loss: 3.5128 - age_loss: 3.2041 - gender_loss: 0.2182 - age_acc: 0.1211 - gende